# Systems of more than one variable

Almost all the problems that we will want to solve are systems of more than one variable.  For example, the Navier-Stokes equation for fluid flow, $C^0$-conforming discretisations of the Cahn-Hilliard equation for phase separation, or systems that coupled different physical processes.

Firedrake supports specifying and solving such systems in *mixed* function spaces.  Here we will use a dual formulation of the Poisson equation as a simple introductory example.

These systems of equations have a block algebraic form, and we will also look at some of the support for efficient preconditioning strategies that Firedrake offers.

## Problem setup
Recall the primal form of the Poisson equation.  Find $u$ satisfying

$$
\begin{align}
\nabla^2 u &= -f \quad\text{in $\Omega$} \\
   u &= u_0 \quad\text{on $\Gamma_D$}\\
   \nabla u \cdot n &= g \quad\text{on $\Gamma_N$.}
   \end{align}$$
   
We obtain the dual form by introducing a vector-valued flux $\sigma = \nabla u$.  Substituting, the problem becomes.  Find $(\sigma, u)$ satisfying

$$
\begin{align}
\sigma - \nabla u &= 0 \quad \text{in $\Omega$} \\
\nabla \cdot \sigma &= -f \quad \text{in $\Omega$} \\
u &= u_0 \quad \text{on $\Gamma_D$}\\
\sigma \cdot n &= g \quad\text{on $\Gamma \setminus \Gamma_D =: \Gamma_N$.}
\end{align}
  $$
  
For the variational formulation, we introduce a pair of function spaces, $\Sigma$ and $V$, and seek $(\sigma, u) \in \Sigma \times V$ such that:

$$
\begin{align}
\int_\Omega (\sigma \cdot \tau + \nabla \cdot \tau u)\,\text{d} x &= 
  \int_\Gamma \tau \cdot n u\,\text{d}s \quad \forall \tau \in \Sigma\\ 
\int_\Omega (\nabla\cdot\sigma)v\,\text{d} x &= - \int_\Omega f v\,\text{d}x 
\quad \forall v \in V.
\end{align}
$$

Notice how the weak condition in the primal form turns into a strong condition on the auxiliary variable $\sigma$ and the strong condition on $u$ in the primal form appears as a weak condition on $\sigma$.

For this problem we will solve with homogeneous Dirichlet (strong) conditions $u_0 = 0$ on the boundary of a square domain $\Omega = [0, 1]^2$, and will choose $f = 10\exp(-100((x-0.5)^2 + (y-0.5)^2))$.

As should be familiar by now, we start by importing the Firedrake package and setting up the discrete problem domain.

In [ ]:
%matplotlib notebook
from firedrake import *

N = 20

mesh = UnitSquareMesh(N, N)

We need to choose an LBB-stable finite element pair for $W = V\times\Sigma$, one choice is the Raviart-Thomas space of degree $k$ for $\Sigma$ and discontinuous polynomials of degree $k$ for $V$, here we choose $k = 1$.

In [ ]:
Sigma = FunctionSpace(mesh, "Raviart-Thomas", 2)
V = FunctionSpace(mesh, "DG", 1)

Note that Firedrake inherits the convention from FEniCS that the specified element degree is the degree of the smallest polynomial space that spans the space of the element.  The lowest order Raviart-Thomas element contains all the constants and some linear functions, hence we specify degree 1.

Now we build the mixed finite element space $W$, and create test and trial functions.

In [ ]:
W = Sigma * V

sigma, u = TrialFunctions(W)
tau, v = TestFunctions(W)

The forcing function $f$ is created as an expression, and we're ready to set up the variational problem.

In [ ]:
x, y = SpatialCoordinate(mesh)
f = 10*exp(-100*((x - 0.5)**2 + (y - 0.5)**2))

a = dot(sigma, tau)*dx + div(tau)*u*dx + div(sigma)*v*dx
L = -f*v*dx

We need a solution variable, and then we can solve the problem. The problem is symmetric, but indefinite, so we can use the MINRES Krylov method.

In [ ]:
wh = Function(W)
solve(a == L, wh, solver_parameters={"ksp_type": "minres"})

Let's have a look at the solution

In [ ]:
sigmah, uh = wh.split()
plot(sigmah);
plot(uh, num_sample_points=1);

## Improving the solver performance

Most problems you'll encounter will require effective *preconditioning* to achieve mesh and parameter *independent* convergence.  By this we mean that the number of iterations to obtain the solution is independent of both the mesh refinement, and hopefully also, any coefficient variability.  Here, we have a constant-coefficient problem, so we only need to worry about mesh refinement.

If we had a primal Poisson problem, we could treat the issue quite easily using an algebraic multigrid preconditioner such as hypre.  However, for this dual formulation is slightly more complicated.

First, we'll illustrate the issue, then look at some possible solutions.

We're going to be running the same problem over and over again at a range of mesh resolutions, so to avoid writing the code again and again we'll wrap the problem up in a function.

In [ ]:
def create_solver(N, solver_parameters, aP=None):
    mesh = UnitSquareMesh(N, N)
    Sigma = FunctionSpace(mesh, "Raviart-Thomas", 2)
    V = FunctionSpace(mesh, "DG", 1)
    W = Sigma * V

    sigma, u = TrialFunctions(W)
    tau, v = TestFunctions(W)
    x, y = SpatialCoordinate(mesh)
    f = 10*exp(-100*((x - 0.5)**2 + (y - 0.5)**2))

    a = dot(sigma, tau)*dx + div(tau)*u*dx + div(sigma)*v*dx
    L = -f*v*dx
    
    wh = Function(W)
    if aP is not None:
        aP = aP(W)
    problem = LinearVariationalProblem(a, L, wh, aP=aP)
    solver = LinearVariationalSolver(problem, solver_parameters=solver_parameters)
    return solver

In [ ]:
parameters = {"ksp_type": "minres", "mat_type": "aij", "pc_type": "icc"}
from firedrake.solving_utils import KSPReasons
for N in [5, 10, 25, 50]:
    solver = create_solver(N, parameters)
    solver.solve()
    print "N = {}, iterations = {}, converged reason = {}".format(
        N, solver.snes.ksp.getIterationNumber(), 
        KSPReasons[solver.snes.ksp.getConvergedReason()])

### Schur complement approaches

Many state of the art solution methods for systems of equations rely on block factorisation and block inverses to construct good preconditioners.  Firedrake supports these through PETSc's `fieldsplit` preconditioning type.

Our system is a block matrix:

$$
\mathcal{A} = \begin{split}\left(\begin{matrix} A & B \\ C & 0 \end{matrix}\right)\end{split}
$$

admitting a factorisation

$$
\begin{split}\left(\begin{matrix} I & 0 \\ C A^{-1} & I\end{matrix}\right)
\left(\begin{matrix}A & 0 \\ 0 & S\end{matrix}\right)
\left(\begin{matrix} I & A^{-1} B \\ 0 & I\end{matrix}\right),\end{split}
$$

with $S = -C A^{-1} B$ the *Schur complement*.  This has an inverse:

$$
\begin{split}\left(\begin{matrix} I & -A^{-1}B \\ 0 & I \end{matrix}\right)
\left(\begin{matrix} A^{-1} & 0 \\ 0 & S^{-1}\end{matrix}\right)
\left(\begin{matrix} I & 0 \\ -CA^{-1} & I\end{matrix}\right).\end{split}
$$

In particular, note that if we drop some of the terms in the factorisation and write

$$ 
\mathcal{P} = \begin{split}\left(\begin{matrix} A & 0 \\ 0 & -C A^{-1} B\end{matrix}\right)\end{split},
$$

then the system $\mathcal{P}^{-1}\mathcal{A}$ has at most four distinct eigenvalues, and so a Krylov method applied to this preconditioned system will converge quickly [Murphy, Golub, and Wathen (2000)](https://doi.org/10.1137/S1064827599355153).

This approach therefore reduces the problem of finding an inverse of $\mathcal{A}$ into the (hopefully simpler) problem of finding good inverses for blocks thereof.

Let's try this out on our problem:

In [ ]:
parameters = {"ksp_type": "minres", 
              "mat_type": "aij", 
              "pc_type": "fieldsplit",
              "pc_fieldsplit_type": "schur",
              "pc_fieldsplit_schur_fact_type": "diag",
              "fieldsplit_0_ksp_type": "cg",
              "fieldsplit_0_pc_type": "icc",
              "fieldsplit_0_ksp_rtol": 1e-12,
              "fieldsplit_1_ksp_type": "cg",
              "fieldsplit_1_pc_type": "none",
              "fieldsplit_1_ksp_rtol": 1e-12}
from firedrake.solving_utils import KSPReasons
for N in [5, 10, 25, 50]:
    solver = create_solver(N, parameters)
    solver.solve()
    print "N = {}, iterations = {}, converged reason = {}".format(
        N, solver.snes.ksp.getIterationNumber(), 
        KSPReasons[solver.snes.ksp.getConvergedReason()])

Great, we got good algorithmic convergence.  Unfortunately, this is still very slow.  This is because the Schur complement operator $S$ is effectively an elliptic operator and we're inverting it with an unpreconditioned Krylov method.

## Exercise

Determine how the iterations required to invert the $S$ change when the mesh size is increased.

- Hint 1: You can ask PETSc to print the convergence status of a solve by providing the solver parameter `"ksp_converged_reason": True`
- Hint 2: The inversion of $S$ is carried out by the `fieldsplit_1_` KSP, so you'll want to print the convergence status of that subsolve.

## Providing an approximation to $S$

Our problem is that we haven't provided a preconditioning matrix to invert $S$.  If $A$ is such that $\text{diag}(A)^{-1}$ is a good approximation to $A^{-1}$, then a sparse approximation $\tilde{S} = -C \text{diag}(A)^{-1} B$ is a good approximation to $S$.  PETSc allows us to create this purely using options:

In [ ]:
parameters = {"ksp_type": "minres", 
              "mat_type": "aij", 
              "pc_type": "fieldsplit",
              "pc_fieldsplit_type": "schur",
              "pc_fieldsplit_schur_fact_type": "diag",
              # Construct \tilde{S} using C diag(A)^{-1} B
              "pc_fieldsplit_schur_precondition": "selfp",
              "fieldsplit_0_ksp_type": "cg",
              "fieldsplit_0_pc_type": "icc",
              "fieldsplit_0_ksp_rtol": 1e-12,
              "fieldsplit_1_ksp_type": "cg",
              "fieldsplit_1_ksp_converged_reason": True,
              # Use algebraic multigrid on \tilde{S}
              "fieldsplit_1_pc_type": "hypre",
              "fieldsplit_1_ksp_rtol": 1e-12}
from firedrake.solving_utils import KSPReasons
for N in [5, 10, 25, 50, 100]:
    solver = create_solver(N, parameters)
    solver.solve()
    print "N = {}, iterations = {}, converged reason = {}".format(
        N, solver.snes.ksp.getIterationNumber(), 
        KSPReasons[solver.snes.ksp.getConvergedReason()])

We could now think about reducing the tolerance on the inner solves, reducing the accuracy of the approximation inverses.  For example, approximating $A^{-1}$ and $S^{-1}$ purely by the application of the preconditioner.

In [ ]:
parameters = {"ksp_type": "minres", 
              "mat_type": "aij", 
              "pc_type": "fieldsplit",
              "pc_fieldsplit_type": "schur",
              "pc_fieldsplit_schur_fact_type": "diag",
              # Construct \tilde{S} using C diag(A)^{-1} B
              "pc_fieldsplit_schur_precondition": "selfp",
              "fieldsplit_0_ksp_type": "preonly",
              "fieldsplit_0_pc_type": "icc",
              "fieldsplit_1_ksp_type": "preonly",
              # Use algebraic multigrid on \tilde{S}
              "fieldsplit_1_pc_type": "hypre"}
from firedrake.solving_utils import KSPReasons
for N in [5, 10, 25, 50, 100]:
    solver = create_solver(N, parameters)
    solver.solve()
    print "N = {}, iterations = {}, converged reason = {}".format(
        N, solver.snes.ksp.getIterationNumber(), 
        KSPReasons[solver.snes.ksp.getConvergedReason()])

## Exercise

See how fast you can solve the problem with $N = 50$.  Is it best to reduce the number of inner iterations, or the number of outer iterations?  What about when $N = 200$.

- Hint 1: For an exact inverse, say `"pc_type": "lu"`.  You can control the accuracy of the inner solves with `ksp_rtol` or `ksp_max_it`.

- Hint 2:  Remember that the "magic" `%timeit` can time execution of a Python expression.  

- Hint 3: For some more ideas, you could look at the [extended Firedrake demo on this topic](http://firedrakeproject.org/demos/saddle_point_systems.py.html).